# Plumbing : Going Deep With Gluon

创建神经网络的两种方式：

## 1.使用``gluon.Block``

In [1]:
import mxnet as mx

from mxnet import nd
from mxnet import gluon
from mxnet import autograd
mx.random.seed(1)

import utils

ctx = mx.cpu()

In [2]:
batch_size = 64
train_data, test_data = utils.load_dataset(batch_size, data_type='mnist')

for data, _ in train_data:
    data = data.as_in_context(ctx)
    print(data.shape)
    break

(64, 1, 28, 28)


In [3]:
class MLP(gluon.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense0 = gluon.nn.Dense(128, activation='relu')
        self.dense1 = gluon.nn.Dense(64, activation='relu')
        self.dense2 = gluon.nn.Dense(10)
        
    def forward(self, X):
        h1 = self.dense0(X)
        h2 = self.dense1(h1)
        return self.dense2(h2)

In [4]:
# 此时输入的维度还没有推测出来(shape inference)
net1 = MLP()
net1.initialize(ctx=ctx)
net1.collect_params

<bound method Block.collect_params of MLP(
  (dense2): Dense(None -> 10, linear)
  (dense1): Dense(None -> 64, Activation(relu))
  (dense0): Dense(None -> 128, Activation(relu))
)>

In [5]:
net1(data)[:1]


[[ 0.00417079 -0.04057368 -0.01482122  0.02092639 -0.01289647 -0.00897914
  -0.02665747 -0.02111475  0.03314075  0.01809338]]
<NDArray 1x10 @cpu(0)>

In [6]:
# 直到有数据传递的时候，shape inference才起作用
net1.collect_params

<bound method Block.collect_params of MLP(
  (dense2): Dense(64 -> 10, linear)
  (dense1): Dense(128 -> 64, Activation(relu))
  (dense0): Dense(784 -> 128, Activation(relu))
)>

In [7]:
net1.name

'mlp0'

## 2.使用``gluon.nn.Sequential``

In [8]:
net2 = gluon.nn.Sequential()
with net2.name_scope():
    net2.add(gluon.nn.Dense(128, activation="relu"))
    net2.add(gluon.nn.Dense(64, activation="relu"))
    net2.add(gluon.nn.Dense(10))

In [9]:
net2.collect_params().initialize(ctx=ctx, force_reinit=True)

In [10]:
net2.collect_params

<bound method Block.collect_params of Sequential(
  (0): Dense(None -> 128, Activation(relu))
  (1): Dense(None -> 64, Activation(relu))
  (2): Dense(None -> 10, linear)
)>

我们可以通过``in_units``来指定输入维度，这样MXNet就不会进行shape inference了

In [11]:
net3 = gluon.nn.Sequential()
with net3.name_scope():
    net3.add(gluon.nn.Dense(128, in_units=784, activation="relu"))
    net3.add(gluon.nn.Dense(64, in_units=128, activation="relu"))
    net3.add(gluon.nn.Dense(10))